In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from fastapi import FastAPI, UploadFile,BackgroundTasks
from embedder import Embedder
from summarizer import Summarizer
from parser import Parser
from database import VDb as database

# app = FastAPI()
parser = Parser()

In [2]:
model = SentenceTransformer("google/embeddinggemma-300m")
vector_store = InMemoryVectorStore(embedding=model)

In [16]:
from rq import Queue
from redis import Redis
redis_con = Redis(port=7777)
que=Queue(connection=redis_con)

In [ ]:
def prep_pdf(file):
    paragraphs, metadata = que.enqueue(parser.extract_paragraphs, file)
    print(f'Doc : {metadata}\n')
    print(f'Total paragraphs[{len(paragraphs)}]\n')
    chunks = que.enqueue(parser.chunk_pdf, paragraphs)
    # compute and upload vector
    print(f'Total chunks[{len(chunks)}]\n')
    database.upload_document_vector(
        metadata=metadata,
        chunks=chunks
    )

In [29]:
@app.post('/upload')
async def upload_pdf(file:UploadFile, background_tasks: BackgroundTasks):
    background_tasks.add_task(prep_pdf, file)
    return {
        "success":True,
        "data": "Pdf uploaded"
    }

@app.get('/query')
async def query(q:str):
    return {
        "success":True,
        "data":{
            database.similarity(q)
        }
    }

In [34]:
from fastapi import UploadFile

file = UploadFile("./STT205.pdf")
file

UploadFile(filename=None, size=None, headers=Headers({}))

In [35]:
import re